In [1]:
# Imports
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tqdm.notebook import tqdm
np.set_printoptions(linewidth=200, suppress=True, formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [2]:
# Load in data
# rootdir = '/content/drive/MyDrive/Extracurriculars/Summer_2021/Polygence/Data/OASIS/csv_files/oasis_3.csv'
rootdir = '/hdd/Polygence/Data/OASIS/csv_files/oasis_3.csv'
df = pd.read_csv(rootdir)
print(df.shape)
df.head()

(2168, 22)


,Subject,MR ID,id,Age,M/F,dx1,mmse,cdr,apoe,TOTAL_HIPPOCAMPUS_VOLUME,...,rhCortexVol,CortexVol,SubCortGrayVol,TotalGrayVol,SupraTentorialVol,lhCorticalWhiteMatterVol,rhCorticalWhiteMatterVol,CorticalWhiteMatterVol,L.SurfArea,R.SurfArea
0,OAS30001,OAS30001_MR_d3132,OAS30001_Freesurfer53_d3132,73.000,F,Cognitively normal,30.000,0.000,23.000,"6,861.900",...,"178,031.559","359,975.258","48,400.000","491,102.258","773,671.599","174,372.329","173,244.012","347,616.342","67,598.100","67,185.800"
1,OAS30001,OAS30001_MR_d0129,OAS30001_Freesurfer53_d0129,65.000,F,Cognitively normal,30.000,0.000,23.000,"7,678.900",...,"187,528.786","379,446.180","50,687.000","517,683.180","810,585.114","184,600.488","182,662.445","367,262.933","70,168.100","69,483.800"
2,OAS30001,OAS30001_MR_d2430,OAS30001_Freesurfer53_d2430,71.000,F,Cognitively normal,30.000,0.000,23.000,"7,105.900",...,"178,872.680","357,784.490","49,058.000","487,405.490","777,931.271","175,955.969","178,172.813","354,128.782","67,905.700","68,000.200"
3,OAS30001,OAS30001_MR_d0757,OAS30001_Freesurfer53_d0757,67.000,F,Cognitively normal,29.000,0.000,23.000,"7,648.200",...,"177,566.875","362,040.151","50,071.000","500,699.151","799,341.921","185,224.780","188,151.990","373,376.770","69,142.300","68,558.800"
4,OAS30002,OAS30002_MR_d2345,OAS30002_Freesurfer53_d2345,73.000,M,Cognitively normal,29.000,0.000,34.000,"7,833.200",...,"230,240.533","457,342.036","56,773.000","607,473.036","1,051,713.751","239,168.338","245,361.377","484,529.716","83,138.100","85,742.300"


In [3]:
# Data pre-processing
df = df.dropna(axis=1, how='all') # Drop any empty columns
df = df.dropna(axis=0, how='any') # Drop any rows with empty values 
df = df.rename(columns={'id':'Freesurfer ID', 'dx1':'Diagnosis', 
                        'TOTAL_HIPPOCAMPUS_VOLUME':'TotalHippocampusVol'}) # Rename columns
# df = df.drop_duplicates(subset='Subject', keep='first') # Keep only the first visit; this is possible because
#                                                         # df is sorted by age
df = df.reset_index(drop=True) # Reset the index
cols = df.columns.tolist()
cols[2], cols[4] = cols[4], cols[2]
df = df[cols]
df.loc[df['cdr'] < 0.5, 'Diagnosis'] = 0
df.loc[~(df['cdr'] < 0.5), 'Diagnosis'] = 1
df = df.drop(['MR ID', 'Freesurfer ID', 'cdr', 'M/F'], axis=1) # Drop categorical and redundant columns
print(df.shape)

(1956, 18)


In [4]:
index = df[df['Diagnosis'] == 0].index

In [5]:
df.drop(index[:37], inplace=True)

In [6]:
df.shape

(1919, 18)

In [7]:
df['age_bins'] = pd.cut(x=df['Age'], bins=[40, 60, 80, 100])

In [8]:
df['Diagnosis'].value_counts()

0    1495
1     424
Name: Diagnosis, dtype: int64

In [9]:
df['age_bins'].value_counts()

(60, 80]     1356
(40, 60]      315
(80, 100]     248
Name: age_bins, dtype: int64

In [10]:
# Load in data
# rootdir = '/content/drive/MyDrive/Extracurriculars/Summer_2021/Polygence/Data/OASIS/csv_files/oasis_3.csv'
rootdir = '/hdd/Polygence/Data/ADNI/patients.csv'
df = pd.read_csv(rootdir)
print(df.shape)
df.head()

(5375, 12)


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I431036,941_S_4764,EMCI,F,85,30,MRI,MPRAGE,Original,6/16/2014,DCM,8/07/2021
1,I307653,941_S_4764,EMCI,F,83,22,MRI,MPRAGE,Original,6/01/2012,DCM,8/07/2021
2,I376064,941_S_4764,EMCI,F,84,28,MRI,MPRAGE,Original,6/10/2013,DCM,8/07/2021
3,I325119,941_S_4764,EMCI,F,83,24,MRI,MPRAGE,Original,8/20/2012,DCM,8/07/2021
4,I294080,941_S_4420,EMCI,M,81,22,MRI,MPRAGE,Original,3/28/2012,DCM,8/07/2021


In [11]:
df['Group'].value_counts()

CN      1521
MCI     1275
EMCI    1224
AD       718
LMCI     637
Name: Group, dtype: int64

In [12]:
df['age_bins'] = pd.cut(x=df['Age'], bins=[40, 60, 80, 100])

In [13]:
df['age_bins'].value_counts()

(60, 80]     3935
(80, 100]    1269
(40, 60]      171
Name: age_bins, dtype: int64